## Imports

In [159]:
import pandas as pd
import numpy as np
import shutil
import psutil
import os


## Global Variables

In [180]:
max_tuples = 100000
# I added default variables to the dictionary
vars = {}
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss  # in bytes
vars_directory =  "/Users/jdanninger/Documents/GitHub/Large-CSV-Querier/Variables/"

## Code

In [ ]:
def read_line(line, line_num):
    try:
        if breakDown(line) is None:
            print("Error: Line is not in the correct format on line: " + str(line_num))
            return
        var_name, command, args = breakDown(line)
        # make case/switch statement for command
        match command:
            case "LoadCSV":
                load(var_name, args)
            case "SetMaxTuples":
                max_tuples = int(args)
            case "Project":
                project(var_name, args)
            case "Select":
                select(var_name, args)
            case "HashJoin":
                merge(var_name, args)
            case "Export":
                export(var_name, args)
            case _:
                print("Error: Command not recognized")
                
    except Exception as e:
        print("Error: " + str(e) + " on line: " + str(line_num))
        return

def export(var_name, args):
    input_var = get_var(args.split(",")[0].strip())
    output_path = args.split(",")[1].strip()
    shutil.copy(input_var, output_path)
    print("Exported " + input_var + " to " + output_path)

def project(var_name, args):
    #args will be in format path, col1, col2, col3
    input_path = get_var(args.split(",")[0].strip())
    temp_path = vars_directory + "temp.csv"
    output_path = get_var(var_name)
    cols = args.split(",")[1:]
    # strip all of cols
    cols = [col.strip() for col in cols]

    num_rows = get_num_rows(input_path)
    curr_row = 0


    while(curr_row < num_rows):
        df = read_csv_with_header(input_path, curr_row, max_tuples) 
        df = df[cols]

        df.to_csv(temp_path, mode='a', header = False, index=False)
        curr_row += max_tuples    

    shutil.move(temp_path, output_path)    


def merge(var_name, args):
    #args will be in format path1, path2
    input1 = get_var(args.split(",")[0].strip())
    input2 = get_var(args.split(",")[1].strip())
    on2 = args.split(",")[2].strip()
    local_max = int(max_tuples/2)
    temp_path = vars_directory + "temp.csv"
    output_path = get_var(var_name)

    num_rows1 = get_num_rows(input1)
    num_rows2 = get_num_rows(input2)
    curr_row1 = 1
    while(curr_row1 <= num_rows1):
        curr_row2 = 1
        while(curr_row2 <= num_rows2):
            df1 = read_csv_with_header(input1, curr_row1, local_max)
            df2 = read_csv_with_header(input2, curr_row2, local_max)
            df = pd.merge(df1, df2, on=on2, how='inner')
            df.to_csv(temp_path, mode='a', header = False, index=False)
            
            curr_row2 += local_max
        print(f"Memory usage: {get_memory_usage() / (1024 * 1024):.2f} MB")
        print("current row is " + str(curr_row1))
        curr_row1 += local_max
    shutil.move(temp_path, output_path)
    

def select(var_name, args):
    input_var = args.split(",")[0].strip()
    input_path = get_var(input_var)
    temp_path = vars_directory + "temp.csv"
    output_path = get_var(var_name)
    filter_statement = args.split(",")[1].strip()
    num_rows = get_num_rows(input_path)
    curr_row = 1

    while(curr_row < num_rows):
        df = read_csv_with_header(input_path, curr_row, max_tuples)
        df =  df[df.apply(lambda row: eval(filter_statement), axis=1)]
        df.to_csv(temp_path, mode='a', header = False, index=False)
        curr_row += max_tuples
    shutil.move(temp_path, output_path)  


def read_csv_with_header(file_path, skiprows, nrows):
    # Read the header first
    header = pd.read_csv(file_path, nrows=1).columns.tolist()
    # Read the data, skipping the specified rows but keeping the header
    data = pd.read_csv(file_path, skiprows=skiprows, nrows=nrows, header=None, names=header)
    return data
    
def get_var(var_name):
    if var_name in vars:
        return vars[var_name]
    else:
        vars[var_name] = vars_directory + var_name + ".csv"
        return vars[var_name]

def load(var_name, source_path):
    # copy the file into /Variables folder and rename it to the variable name
    destination_path = vars_directory + var_name + ".csv"
    vars[var_name] = destination_path
    shutil.copy(source_path, destination_path)
    print("File copied to " + destination_path)
    print("file from " + source_path + " loaded into " + var_name)

def get_num_rows(file_path):
    with open(file_path, "r") as file:
        num_rows = sum(1 for line in file)
    return num_rows

def breakDown(line):
    # Breaks down the string into the variable name, command, and arguments
    # Returns a tuple of the three or None if the string is not in the correct format
    var_name = ""
    command = ""
    args = ""
    eq_split = line.split("=")
    # Check if the line has an equals sign
    if len(eq_split) < 2:
        return None
    var_name = eq_split[0].strip()
    command_split = eq_split[1].split("{")
    if len(command_split) < 2:
        return None
    command = command_split[0].strip()
    args = command_split[1].strip()[:-1]
    return (var_name, command, args)



In [ ]:
# script_location = input("Enter the location of the script: ")
script_location = "/Users/jdanninger/Documents/GitHub/Large-CSV-Querier/stresstest.txt"
with open(script_location, 'r') as file:
    line_no = 1
    for line in file:
        read_line(line, line_no)
        line_no += 1

File copied to /Users/jdanninger/Documents/GitHub/Large-CSV-Querier/Variables/relationships.csv
file from /Users/jdanninger/Documents/GitHub/Large-CSV-Querier/OffshoreData/relationships.csv loaded into relationships
File copied to /Users/jdanninger/Documents/GitHub/Large-CSV-Querier/Variables/entities.csv
file from /Users/jdanninger/Documents/GitHub/Large-CSV-Querier/OffshoreData/nodes-entities.csv loaded into entities


/var/folders/cn/1fh6dg9d5rxf80xmd9jhmq0h0000gn/T/ipykernel_62915/1883075417.py:103: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, skiprows=skiprows, nrows=nrows, header=None, names=header)


Memory usage: 510.20 MB
Memory usage: 505.23 MB
Memory usage: 496.83 MB


/var/folders/cn/1fh6dg9d5rxf80xmd9jhmq0h0000gn/T/ipykernel_62915/1883075417.py:103: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, skiprows=skiprows, nrows=nrows, header=None, names=header)


Memory usage: 486.16 MB


...

Memory usage: 490.67 MB


/var/folders/cn/1fh6dg9d5rxf80xmd9jhmq0h0000gn/T/ipykernel_62915/1883075417.py:103: DtypeWarning: Columns (2,3,10,11,12,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, skiprows=skiprows, nrows=nrows, header=None, names=header)


Memory usage: 513.23 MB
Memory usage: 513.27 MB


/var/folders/cn/1fh6dg9d5rxf80xmd9jhmq0h0000gn/T/ipykernel_62915/1883075417.py:103: DtypeWarning: Columns (6,7,12,13,14,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, skiprows=skiprows, nrows=nrows, header=None, names=header)


Memory usage: 513.31 MB
Memory usage: 529.34 MB
